In [3]:
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json

In [4]:
def load_data(path, idx, p_success=1.0):
    filename = 'n_subbins='+str(int(idx))+f'_p_pi_suc={p_success:.5f}.csv'
    df = pd.read_csv(path + filename)
    return df

In [5]:
path_root='./data/raw/'
path_results = './data/'
path_est=('aml/', 'hmm/', 'hmm_no_exp/', 'minbins/', 'entropy_gain/')
path_test=('beryllium/', 'ideal_bk/', 'pi_success/')
path_end=('bright_R_D_0/', 'bright_R_D_0_5/', 'dark_R_D_0/', 'dark_R_D_0_5/')
n_estimators = len(path_est)
n_subbins = np.arange(5, 36)
t_detection_max = 200 # us

n_subbins_histogram = 20
#
n_subbins_avg = np.zeros((len(n_subbins)))
t_detection_avg = np.zeros((len(n_subbins)))
n_errors = np.zeros((len(n_subbins)))
n_pi_pulse_avg = np.zeros((len(n_subbins)))
histogram_nsubbin = np.zeros((n_subbins_histogram))

In [4]:
for test_idx in range(2):
    df_results = pd.DataFrame()
    df_histogram = pd.DataFrame()
    for path_end_idx in range(len(path_end)):
        for l, estimator_directory in enumerate(path_est):
            path = path_root + estimator_directory + path_test[test_idx] + path_end[path_end_idx]
            for i, j in enumerate(n_subbins):
                df = load_data(path, j)     
                n_subbins_avg_i = np.mean(df['n_subbins_0'].to_numpy())
                t_detection_avg_i = (200.0 / j) * n_subbins_avg_i

                if (path_end_idx > 1):
                    n_errors_i = len(df['prediction_0'].to_numpy()) - np.sum(df['prediction_0'].to_numpy())
                else:
                    n_errors_i = np.sum(df['prediction_0'].to_numpy())

                n_subbins_avg[i] = n_subbins_avg_i
                t_detection_avg[i] = t_detection_avg_i
                n_errors[i] = n_errors_i
                if (l == 0):
                    n_pi_pulse_avg[i] = 0
                else:
                    n_pi_pulse_avg[i] = np.mean(df['n_pi_pulses_0'].to_numpy())
                
                # include the histogram thingy
                if (j == n_subbins_histogram):
                    unique, frequency = np.unique(df['n_subbins_0'].to_numpy(), return_counts = True)
                    histogram_nsubbin[(unique-1).astype(int)] = frequency
                    df_histogram.insert(loc=0, column='hist/'+estimator_directory+path_end[path_end_idx], value=histogram_nsubbin)
                    histogram_nsubbin = np.zeros((n_subbins_histogram))
                    
            df_results.insert(loc=0, column='t_avg/'+estimator_directory+path_end[path_end_idx], value=t_detection_avg)
            df_results.insert(loc=0, column='n_errors/'+estimator_directory+path_end[path_end_idx], value=n_errors)
            df_results.insert(loc=0, column='n_pi_avg/'+estimator_directory+path_end[path_end_idx], value=n_pi_pulse_avg)
    df_histogram.insert(loc=0, column='idx', value=np.arange(1, n_subbins_histogram+1, 1))
    df_results.insert(loc=0, column='n_subbins', value=n_subbins)
    df_results.to_csv(path_results + path_test[test_idx][:-1]+'.csv')
    df_histogram.to_csv(path_results + path_test[test_idx][:-1]+'_histogram_n=' + str(n_subbins_histogram) +'.csv')

In [5]:
test_idx = 2
p_success = np.arange(0.995, 1, 0.0005)
n_subbins = 20

n_subbins_avg = np.zeros((len(p_success)))
t_detection_avg = np.zeros((len(p_success)))
fidelity = np.zeros((len(p_success)))
n_pi_pulse_avg = np.zeros((len(p_success)))

df_results = pd.DataFrame()

for path_end_idx in range(len(path_end)):
    for l, estimator_directory in enumerate(path_est):
        if (l==0):
            continue # the AML estimator does not apply Pi-pulses...
        path = path_root + estimator_directory + path_test[test_idx] + path_end[path_end_idx]
        for i, j in enumerate(p_success):
                df = load_data(path, n_subbins, p_success=j)     
                n_subbins_avg_i = np.mean(df['n_subbins_0'].to_numpy())
                t_detection_avg_i = (200.0 / n_subbins) * n_subbins_avg_i

                if (path_end_idx > 1):
                    fidelity_i =  np.sum(df['prediction_0'].to_numpy()) / len(df['prediction_0'].to_numpy())
                else:
                    fidelity_i = (len(df['prediction_0'].to_numpy()) - np.sum(df['prediction_0'].to_numpy()))/len(df['prediction_0'].to_numpy())

                n_subbins_avg[i] = n_subbins_avg_i
                t_detection_avg[i] = t_detection_avg_i
                fidelity[i] = fidelity_i
                n_pi_pulse_avg[i] = np.mean(df['n_pi_pulses_0'].to_numpy())
                
        df_results.insert(loc=0, column='t_avg/'+estimator_directory+path_end[path_end_idx], value=t_detection_avg)
        df_results.insert(loc=0, column='fidelity/'+estimator_directory+path_end[path_end_idx], value=fidelity)
        df_results.insert(loc=0, column='n_pi_avg/'+estimator_directory+path_end[path_end_idx], value=n_pi_pulse_avg)
                    
df_results.insert(loc=0, column='p_success', value=p_success)
df_results.to_csv(path_results + path_test[test_idx][:-1]+'.csv')

In [6]:
def load_trajectory(path, idx, p_success=1.0):
    filename = 'n_subbins='+str(int(idx))+f'_p_pi_suc={p_success:.5f}.json'
    # Opening trajectory JSON file
    f = open(path+filename)
    binned_traj = json.loads(json.load(f))
    return binned_traj

In [13]:
n_subbins = 20
n_runs = 2e4
path_est=('minbins/', 'entropy_gain/', 'hmm/')
path_traj = 'traj_analysis/'
path_end = ['dark/', 'bright/']



In [18]:
df_histogram = pd.DataFrame()
for path_end_idx in range(len(path_end)):
    for l, estimator_directory in enumerate(path_est):
        path = path_root + estimator_directory + path_traj + path_end[path_end_idx]
        binned_traj = load_trajectory(path, n_subbins)
        n_estimators = np.zeros(n_subbins)
        n_pi_pulses = np.zeros(n_subbins)
        for traj_key in binned_traj.keys():
            n_estimators_i = binned_traj[traj_key].get('probability') * n_runs
            pi_pulses_arr_i = binned_traj[traj_key].get('pi_pulse_0')
            for k, subbin_bool in enumerate(pi_pulses_arr_i):
                n_estimators[k]+=n_estimators_i
                n_pi_pulses[k]+=n_estimators_i*subbin_bool
                
        df_histogram.insert(loc=0, column='n_estimator/'+estimator_directory+path_end[path_end_idx], value=n_estimators)
        df_histogram.insert(loc=0, column='n_pi_pulse/'+estimator_directory+path_end[path_end_idx], value=n_pi_pulses)          
                
df_histogram.insert(loc=0, column='subbin_idx', value=np.arange(1, n_subbins+1, 1))
df_histogram.to_csv(path_results + 'trajectory_analysis_histogram.csv')
df_histogram              

,subbin_idx,n_pi_pulse/hmm/bright/,n_estimator/hmm/bright/,n_pi_pulse/entropy_gain/bright/,n_estimator/entropy_gain/bright/,n_pi_pulse/minbins/bright/,n_estimator/minbins/bright/,n_pi_pulse/hmm/dark/,n_estimator/hmm/dark/,n_pi_pulse/entropy_gain/dark/,n_estimator/entropy_gain/dark/,n_pi_pulse/minbins/dark/,n_estimator/minbins/dark/
0,1,5646.0,20000.0,5628.0,20000.0,5763.0,20000.0,20000.0,20000.0,20000.0,20000.0,20000.0,20000.0
1,2,0.0,5646.0,0.0,5628.0,0.0,5763.0,0.0,20000.0,0.0,20000.0,0.0,20000.0
2,3,5646.0,5646.0,5628.0,5628.0,5763.0,5763.0,1673.0,5742.0,1619.0,5628.0,1657.0,5696.0
3,4,0.0,5646.0,0.0,5628.0,0.0,5763.0,0.0,1673.0,0.0,1619.0,0.0,1657.0
4,5,477.0,1661.0,445.0,1648.0,495.0,1659.0,1673.0,1673.0,1619.0,1619.0,1657.0,1657.0
5,6,0.0,477.0,0.0,445.0,0.0,495.0,0.0,1673.0,0.0,1619.0,0.0,1657.0
6,7,477.0,477.0,445.0,445.0,495.0,495.0,157.0,486.0,120.0,467.0,128.0,454.0
7,8,0.0,477.0,0.0,445.0,0.0,495.0,0.0,157.0,0.0,120.0,0.0,128.0
8,9,35.0,131.0,36.0,129.0,37.0,146.0,157.0,157.0,120.0,120.0,128.0,128.0
9,10,0.0,35.0,0.0,36.0,0.0,37.0,0.0,157.0,0.0,120.0,0.0,128.0
